In [1]:
import tensorflow as tf
import numpy as np
import random
import os
import copy
import time
import keras
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# check if gpu is using
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

In [3]:
# load model
model = keras.models.load_model('./alexnet-cifar10_origin.h5', compile=False)

In [4]:
# load datasets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print(x_train.shape)

(50000, 32, 32, 3)


In [5]:
# get weights
theta = np.array(model.get_weights(), dtype=object)

In [6]:
# return: Population[2]
#     [0]: processed input
#     [1]: processed weight
def initPopulation(x, theta, n, input_mutation_scale, weight_mutation_scale):
    processed_x = x[np.random.randint(len(x), size=n)] # randomly choose n inputs from x
    processed_theta = theta

    # input level
    # save the mutated inputs to the GA object (i.e. self.input_x)
    if input_mutation_scale != 0 and input_mutation_scale != None:
        processed_x = (np.clip((processed_x/255 + np.random.standard_cauchy(processed_x.shape) * input_mutation_scale),0,1) * 255).astype(int)

    # weight level
    # save the mutated weights to the GA object (i.e. self.model_weights)
    if weight_mutation_scale != 0 and weight_mutation_scale != None:
        processed_theta = []
        for layer_weight in theta:
            processed_theta.append(layer_weight + np.random.standard_cauchy(layer_weight.shape) * weight_mutation_scale)

        return [processed_x, processed_theta]

In [7]:
P = initPopulation(x_test[:10], theta, 100, 0.1, 2)

In [8]:
# Return: list of size = len(x)*number of layer
def nanFitness(P, model, phi):
    x = P[0] # n inputs chosen from dataset
    
    # compute immediate outputs from each layer
    extractor = keras.Model(inputs=model.inputs, outputs=[layer.output for layer in model.layers])
    features = extractor.predict(x)
    
    # flatten neurons in the last layer for each input
    last_layer_outputs = features[-1].reshape((len(x), -1))
    
    # normalize neurons
    normalized_outputs = last_layer_outputs/(np.amax(last_layer_outputs) - np.amin(last_layer_outputs))
    
    fitness_values = np.amax(normalized_outputs, axis=1) - np.amin(normalized_outputs, axis=1)
    return fitness_values

In [9]:
from multiprocessing import Process
import os
import numpy as np

In [10]:
class InconsistencyFunc:
    def __init__(self, py_script, backends, model_path, input_path, save_paths):
        self.py_script = py_script
        self.backend_1, self.backend_2 = backends
        self.model_path = model_path
        self.input_path = input_path
        self.save_path_1, self.save_path_2 = save_paths
        self.cmd_1 = f'python {self.py_script} {self.backend_1} {self.model_path} {self.input_path} {self.save_path_1} '
        self.cmd_2 = f'python {self.py_script} {self.backend_2} {self.model_path} {self.input_path} {self.save_path_2} '
        
    def backend_predicts(self, bkd_k, layer_idx):
        if bkd_k == 1:
            os.system(self.cmd_1 + str(layer_idx))
        else:
            os.system(self.cmd_2 + str(layer_idx))

    def get_predictions(self, layer_idx):
        p1 = Process(target=self.backend_predicts, args=(1, layer_idx))
        p2 = Process(target=self.backend_predicts, args=(2, layer_idx))
        
        p1.start()
        p2.start()
        p1.join()
        p2.join()
        
    def computeDiff(self, layer_idx=-1):
        self.get_predictions(layer_idx)
        
        self.predictions_1 = np.load(self.save_path_1)
        self.predictions_2 = np.load(self.save_path_2)
        self.predictions_diff = self.predictions_2 - self.predictions_1
        
        self.output_distance = np.sum(np.sum(self.predictions_diff**2, axis=1)) / len(self.predictions_1)
        
        return self.output_distance
        
        
        

In [11]:
model_path = "my_model.h5"
input_path = "inputs.npy"
save_paths = ['save1.npy', 'save2.npy']
py_script = "get_predicts.py"
backends = ['theano', 'tensorflow']

In [12]:
def inconsistFitness(P, model, phis):
    model_path = "if_model.h5"
    model.save(model_path)
    
    input_path = "if_inputs.npy"
    np.save(input_path, P[0])
    
    save_paths = ['if_output1.npy', 'if_output2.npy']
    py_script = "get_predicts.py"
    backends = phis
    
    incFunc = InconsistencyFunc(py_script, backends, model_path, input_path, save_paths)
    return incFunc.computeDiff()
    

In [13]:
# Population: 
#     [0]: inputs
#     [1]: weights
# phi2 will not be used if nanFitness is used
# Return: list of size = len(x)
def computeFitness(P, model, fitness_func, phis):
    if fitness_func == "nan":
        return nanFitness(P, model, phis[0])
    if fitness_func == "inc":
        return inconsistFitness(P, model, phis)

In [14]:
Fit = computeFitness(P, model, 'nan', ['theano', 'tensorflow'])

In [ ]:
def select(P, m, Fit):
    x = P[0]
    selected_index = np.argpartition(Fit, -m)[-m:]
    selected_x = x[selected_index]
    return selected_x

In [ ]:
P_prime = select(P, 5, Fit)

In [ ]:
def selectParents(P2):
    x1, x2 = random.sample(list(P2), 2)
    return x1, x2

In [ ]:
x1, x2 = selectParents(P_prime)

In [ ]:
plt.imshow(x1)

In [ ]:
plt.imshow(x2)

In [ ]:
# Return 1-D array with length=3072 (32*32*3)
def crossover(x1, x2, r1):
    x1_flatten = x1.flatten()
    x2_flatten = x2.flatten()
    
    # inefficient way
    x_prime = []
    for i in range(len(x1_flatten)):
        rp = random.random()
        if rp > r1:
            x_prime.append(x1_flatten[i])
        else:
            x_prime.append(x2_flatten[i])

    # efficient way
    # use numpy random generate random list
    # ...
    
    return x_prime


In [ ]:
x_prime = crossover(x1, x2, 0.5)

In [ ]:
plt.imshow(np.array(x_prime).reshape((32,32,3)))

In [ ]:
def mutate(x_prime, x_shape, mutation_scale):
    x_prime_np = np.array(x_prime)
    x_2prime=(np.clip((x_prime_np/255 + np.random.standard_cauchy(x_prime_np.shape) * mutation_scale),0,1) * 255).astype(int)
    x_2prime = x_2prime.reshape(x_shape)
    return x_2prime

In [ ]:
x_2prime = mutate(x_prime, (32,32,3), 0)

In [ ]:
plt.imshow(x_2prime)

In [ ]:
def checkFailed(P_2prime, model):
    x = P[0]
    F = []
    for i in x:
        try:
            f.predict(i)
        except:
            F.append(i)
    return F

In [ ]:
# configure parameters
f = model     # A DNN
theta = theta # Weights of the DNN
x = x_test     # input to the DNN
phi = "tensorflow"   # the targeted DL frameworks
r1 = 0.5    # Crossover rate
r2 = 0.1    # Mutation rate
maxIter = 1    # Maximum iterations
m = 5     # Number of selected parents
n = len(x_test)


# Algorithm starts
def algo_1():
    iteration = 0
    F = {}      # A set of failed test casses
    P = initPopulation(f, x, theta, "input", 0.1, 0.2)

    while iteration < maxIter:
        iteration += 1
        Fit = computeFitness(P[0], model, "nan", phi, phi)
        P_prime = select(P, m, Fit)
        P_2prime = P_prime
        while len(P_2prime) < n:
            x1, x2 = selectParents(P_prime)
            x_prime = crossover(x1, x2, r1)
            r = random.uniform(0, 1)
            if r < r2:
                x_2prime = mutate(x_prime, r2)
                P_2prime = P_2prime.union(x_2prime)
        X = checkFailed(P_2prime)
        if X != {}:
            F = F.union(X)

            return F


In [ ]:
F = {}      # A set of failed test casses
P = initPopulation(f, x, theta, "input", 0.1, 0.2)
        

In [ ]:
Fit = computeFitness(P[0], model, "nan", phi, phi)

In [ ]:
max(Fit)

In [ ]:
P_prime = select(P, m, Fit)

In [ ]:
P_2prime = list(np.expand_dims(P_prime, 1))

In [ ]:
start_time = time.perf_counter()
while len(P_2prime) < 10000:
    x1, x2 = selectParents(P_prime)
    x_prime = crossover(x1, x2, r1)
    r = random.uniform(0, 1)
    if r < r2:
        x_2prime = mutate(x_prime, r2, x1.shape)
        P_2prime.append(x_2prime)
end_time = time.perf_counter()
end_time-start_time

In [ ]:
new_P = [P_2prime, P[1]]

In [ ]:
X,D = checkFailed(new_P, f)

In [ ]:
D